In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import json
import pprint
import os
import clip
import imagehash
import cv2
import imagehash
from tqdm import tqdm_notebook as tqdm
from PIL import Image
import pickle
import shutil
import torch
import re
from string import punctuation
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download("stopwords")
from torch.utils.data import Dataset, DataLoader

stopwords_ = set(stopwords.words("english"))



[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("../data/val_data_map.csv")

In [3]:
df.head()

,caption,product,label
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566


In [4]:
df["step_1_lower"] = df["caption"].apply(lambda x:x.lower())
df.head()

,caption,product,label,step_1_lower
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...


In [5]:
df["step_2_no_sp_char"] = df["step_1_lower"].apply(lambda x:re.sub(r"[^A-Za-z0-9\s]+", "", x))
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...


In [6]:
def no_stop(x):
    tokens = word_tokenize(x)
    clean_tokens = [t for t in tokens if not t in stopwords_]
    clean_text = " ".join(clean_tokens)
    return clean_text
df["step_3_no_stop"] = df["step_2_no_sp_char"].apply(no_stop)
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...


In [7]:
df["step_4_no_num"] = df["step_3_no_stop"].apply(lambda x: re.sub(r"\b[0-9]+\b\s*", "", x))
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...


In [8]:
df["step_5_no_dupe"] = df["step_4_no_num"].apply(lambda x:" ".join(set(word_tokenize(x))))
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,luohu arrangement valentines decorative chest ...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,years childrens little old boys baby gift chil...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,tail men fox ting alternative qu sex sai women...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,lucky patron paintings decorative traditional ...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lee jeans mens tide brand slacksmusic lgnace t...


In [9]:
def no_short_token(x):
    tokens = word_tokenize(x)
    clean_tokens = [t for t in tokens if len(t) > 1]
    clean_text = " ".join(clean_tokens)
    return clean_text
df["step_6_no_len_1_token"] = df["step_5_no_dupe"].apply(lambda x:" ".join(set(word_tokenize(x))))
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,luohu arrangement valentines decorative chest ...,arrangement chest valentines decorative nansha...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,years childrens little old boys baby gift chil...,years childrens little old boys baby gift chil...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,tail men fox ting alternative qu sex sai women...,tail men fox metal alternative qu sex sai wome...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,lucky patron paintings decorative traditional ...,lucky patron paintings decorative traditional ...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lee jeans mens tide brand slacksmusic lgnace t...,lee jeans mens tide brand slacksmusic lgnace t...


In [10]:
def crop_token(x):
    tokens = x.split(" ")
    total_len = len(tokens)
    if total_len >3:
        percent_70 = round(total_len*0.7)
        crop_text = tokens[0:percent_70]
        return " ".join(crop_text)
    else:
        return " ".join(tokens)
df["caption_crop_first_70%"] = df["caption"].apply(crop_token)
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token,caption_crop_first_70%
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,luohu arrangement valentines decorative chest ...,arrangement chest valentines decorative nansha...,Tanabata Valentine's Day Shenzhen Bao'an M Nan...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,years childrens little old boys baby gift chil...,years childrens little old boys baby gift chil...,Children's Toys Little Girl over 6 Years Old G...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,tail men fox ting alternative qu sex sai women...,tail men fox metal alternative qu sex sai wome...,sm qing qu Alternative Sex between Men and Wom...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,lucky patron paintings decorative traditional ...,lucky patron paintings decorative traditional ...,Traditional Chinese Painting Burnin' Up Yingke...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lee jeans mens tide brand slacksmusic lgnace t...,lee jeans mens tide brand slacksmusic lgnace t...,lgnace Lee Men's Jeans Thick Section Distresse...


In [11]:
def crop_token(x):
    tokens = x.split(" ")
    total_len = len(tokens)
    if total_len > 3:
        percent_70 = round(total_len*0.7)
        crop_text = tokens[-percent_70:]
        return " ".join(crop_text)
    else:
        return " ".join(tokens)
df["caption_crop_last_70%"] = df["caption"].apply(crop_token)
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token,caption_crop_first_70%,caption_crop_last_70%
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,luohu arrangement valentines decorative chest ...,arrangement chest valentines decorative nansha...,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,M Nanshan Futian Luohu Car Decorative Flowers ...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,years childrens little old boys baby gift chil...,years childrens little old boys baby gift chil...,Children's Toys Little Girl over 6 Years Old G...,Old Girls 3-9 Boys 7 Students I Day Gift 10 Ch...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,tail men fox ting alternative qu sex sai women...,tail men fox metal alternative qu sex sai wome...,sm qing qu Alternative Sex between Men and Wom...,between Men and Women with Metal Tail Fox Tail...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,lucky patron paintings decorative traditional ...,lucky patron paintings decorative traditional ...,Traditional Chinese Painting Burnin' Up Yingke...,Yingkesong Landscape Painting Patron Living Ro...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lee jeans mens tide brand slacksmusic lgnace t...,lee jeans mens tide brand slacksmusic lgnace t...,lgnace Lee Men's Jeans Thick Section Distresse...,Section Distressed Straight Youth Autumn Tide ...


In [12]:
def crop_token(x):
    tokens = x.split(" ")
    total_len = len(tokens)
    if total_len >3:
        percent_70 = round(total_len*0.7)
        strt_idx = (total_len // 2) - (percent_70 // 2)
        end_idx = (total_len // 2) + (percent_70 // 2)
        crop_text = tokens[strt_idx:end_idx+1]
        return " ".join(crop_text)
    else:
        return " ".join(tokens)
df["caption_crop_middle_70%"] = df["caption"].apply(crop_token)
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token,caption_crop_first_70%,caption_crop_last_70%,caption_crop_middle_70%
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,luohu arrangement valentines decorative chest ...,arrangement chest valentines decorative nansha...,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,M Nanshan Futian Luohu Car Decorative Flowers ...,Shenzhen Bao'an M Nanshan Futian Luohu Car Dec...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,years childrens little old boys baby gift chil...,years childrens little old boys baby gift chil...,Children's Toys Little Girl over 6 Years Old G...,Old Girls 3-9 Boys 7 Students I Day Gift 10 Ch...,over 6 Years Old Girls 3-9 Boys 7 Students I D...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,tail men fox ting alternative qu sex sai women...,tail men fox metal alternative qu sex sai wome...,sm qing qu Alternative Sex between Men and Wom...,between Men and Women with Metal Tail Fox Tail...,Alternative Sex between Men and Women with Met...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,lucky patron paintings decorative traditional ...,lucky patron paintings decorative traditional ...,Traditional Chinese Painting Burnin' Up Yingke...,Yingkesong Landscape Painting Patron Living Ro...,Burnin' Up Yingkesong Landscape Painting Patro...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lee jeans mens tide brand slacksmusic lgnace t...,lee jeans mens tide brand slacksmusic lgnace t...,lgnace Lee Men's Jeans Thick Section Distresse...,Section Distressed Straight Youth Autumn Tide ...,Men's Jeans Thick Section Distressed Straight ...


In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-zh")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-zh")
model.to("cuda")

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1

In [14]:
caption_loader  = DataLoader(df["caption"].values,batch_size=64,shuffle=False,num_workers=12)

In [15]:
def translate_to_cn(txt):
    with torch.no_grad():
        token = tokenizer.prepare_seq2seq_batch(txt,return_tensors='pt')
        token = token.to("cuda")
        pred = model.generate(**token)
        pred = pred.detach().cpu()
        return pred


# def translate_to_cn_no_decode(txt):
#     with torch.no_grad():
#         token = tokenizer.prepare_seq2seq_batch(txt,return_tensors='pt')
#         token = token.to("cuda")
#         pred = model.generate(**token)
#         pred = pred.detach().cpu()
#         return pred

# df["caption_cn"] = df["caption"].apply(translate_to_cn)
# df.head()

In [16]:
all_transalated_txt = []
for b in tqdm(caption_loader):
    transalated_txt = translate_to_cn(b)
    transalated_txt = tokenizer.batch_decode(transalated_txt)
    transalated_txt = [t.replace("<pad>","") for t in transalated_txt]
    transalated_txt = np.array(transalated_txt)
    all_transalated_txt.append(transalated_txt)

  0%|          | 0/782 [00:00<?, ?it/s]/home/ubuntu/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3516: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
100%|██████████| 782/782 [4:51:49<00:00, 22.39s/it]  


In [17]:
all_transalated_txt_concat = np.concatenate(all_transalated_txt)

In [18]:
all_transalated_txt_concat.shape

(50000,)

In [19]:
filename = "./translated_caption.pkl"
file = open(filename,"wb")
pickle.dump(all_transalated_txt,file)
file.close()

In [20]:
df["captions_cn"] = all_transalated_txt_concat

In [21]:
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token,caption_crop_first_70%,caption_crop_last_70%,caption_crop_middle_70%,captions_cn
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,luohu arrangement valentines decorative chest ...,arrangement chest valentines decorative nansha...,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,M Nanshan Futian Luohu Car Decorative Flowers ...,Shenzhen Bao'an M Nanshan Futian Luohu Car Dec...,塔那巴塔情人节 清心宝安 南山福田罗虎 汽车装饰花花花花花花 胸衣布置安排
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,years childrens little old boys baby gift chil...,years childrens little old boys baby gift chil...,Children's Toys Little Girl over 6 Years Old G...,Old Girls 3-9 Boys 7 Students I Day Gift 10 Ch...,over 6 Years Old Girls 3-9 Boys 7 Students I D...,6岁以上的儿童玩具小女孩 3-9 男孩 7 学生I日 学生I日 10 儿童8 婴儿4 拼图
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,tail men fox ting alternative qu sex sai women...,tail men fox metal alternative qu sex sai wome...,sm qing qu Alternative Sex between Men and Wom...,between Men and Women with Metal Tail Fox Tail...,Alternative Sex between Men and Women with Met...,使用金属尾狐尾巴的男女替代性行为
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,lucky patron paintings decorative traditional ...,lucky patron paintings decorative traditional ...,Traditional Chinese Painting Burnin' Up Yingke...,Yingkesong Landscape Painting Patron Living Ro...,Burnin' Up Yingkesong Landscape Painting Patro...,中华传统绘画烧在延克松露地绘画主办人 客厅绘画老板办公室 幸运装饰墙壁
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lee jeans mens tide brand slacksmusic lgnace t...,lee jeans mens tide brand slacksmusic lgnace t...,lgnace Lee Men's Jeans Thick Section Distresse...,Section Distressed Straight Youth Autumn Tide ...,Men's Jeans Thick Section Distressed Straight ...,Lgnace Lee Lee Men的Jeans Thick分会受困于直立青年 秋秋潮的B...


In [22]:
df.to_csv("../data/val_data_prompt_clean.csv",index=False)